In [1]:
using LinearAlgebra, Distributions, Combinatorics, Random, Kronecker, SpecialFunctions
include("../DCM_model/BBVI_utils.jl")

# Object for observed data
struct TDCMObs{T <: AbstractFloat}
    # data
    Y           :: Array{Int, 3}
    Q           :: Matrix{Int}
    D           :: Vector{Matrix{Int}}
    U           :: Vector{Vector{Matrix{T}}}
    X           :: Vector{Vector{Matrix{T}}}
    group       :: Vector{Int}
    skill_dict  :: Dict{Int, Vector{Int}}
end

function TDCMObs(
    Y       :: Array{Int, 3}, 
    Q       :: Matrix{Int},
    U       :: Vector{Vector{Matrix{T}}},
    X       :: Vector{Vector{Matrix{T}}},
    group   :: Vector{Int}) where T <: AbstractFloat
    D = generate_delta(Q)
    K, L = size(Q, 2), size(D[1], 1)
    skill_dict = Dict{Int, Vector{Int}}()
    for l in 0:(L - 1)
        skill_dict[l + 1] = reverse(digits(l, base=2, pad=K))
    end
    TDCMObs(Y, Q, D, U, X, group, skill_dict)
end

# Object including latent variables and model parameters
struct TDCModel{T <: AbstractFloat}
    # data
    obs         :: TDCMObs
    # prior distribution parameters
    mu_beta_prior       :: Vector{Vector{T}}
    V_beta_prior        :: Vector{Matrix{T}}
    mu_omega_prior      :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_omega_prior       :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    a_tau_prior         :: Vector{Vector{Vector{Vector{T}}}}
    b_tau_prior         :: Vector{Vector{Vector{Vector{T}}}}
    # This option allocates extra memory based on number of threads availible in the environment
    enable_parallel     :: Bool
    # Variational distribution parameters
    pi_star             :: Vector{Vector{Vector{T}}}
    mu_beta_star        :: Vector{Vector{T}}
    V_beta_star         :: Vector{Matrix{T}}
    mu_gamma_star       :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_gamma_star        :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    mu_omega_star       :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_omega_star        :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    a_tau_star          :: Vector{Vector{Vector{Vector{T}}}}
    b_tau_star          :: Vector{Vector{Vector{Vector{T}}}}
    # Number of samples for noisy gradient
    M                   :: Int
    # Preallocated storage for samples from variational distribution
    Z_sample            :: Vector{Vector{Vector{Vector{Int}}}}
    beta_sample         :: Vector{Vector{Vector{T}}}
    gamma_sample        :: Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}
    omega_sample        :: Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}
    tau_sample          :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    # Preallocated storage for noisy gradient descent calculations
    storage_L           :: Vector{T}
    storage_L2          :: Vector{T}
    storage_L3          :: Vector{T}
    storage_L4          :: Vector{T}
    storage_LL          :: Matrix{T}
    storage_LL2         :: Matrix{T}
    storage_LL3         :: Matrix{T}
    storage_LL4         :: Matrix{T}
    # Preallocated storage for matrix vectorization operations
    storage_comm        :: Matrix{T}
    storage_dup         :: Matrix{T}
    storage_Lsqr        :: Vector{T}
    storage_Lsqr2       :: Vector{T}
    storage_L2L2        :: Matrix{T}
    storage_C           :: Matrix{T}
    storage_gradC       :: Vector{T}
    # Preallocated Identity matrix
    I_LL                :: Matrix{T}
    # Preallocated storage for parallel noisy gradient descent calculations
    storage_L_par       :: Vector{Vector{T}}
    storage_L2_par      :: Vector{Vector{T}}
    storage_L3_par      :: Vector{Vector{T}}
    storage_L4_par      :: Vector{Vector{T}}
    storage_LL_par      :: Vector{Matrix{T}}
    storage_LL2_par     :: Vector{Matrix{T}}
    storage_LL3_par     :: Vector{Matrix{T}}
    storage_LL4_par     :: Vector{Matrix{T}}
    # Preallocated storage for parallel matrix vectorization operations
    storage_comm_par    :: Vector{Matrix{T}}
    storage_dup_par     :: Vector{Matrix{T}}
    storage_Lsqr_par    :: Vector{Vector{T}}
    storage_Lsqr2_par   :: Vector{Vector{T}}
    storage_L2L2_par    :: Vector{Matrix{T}}
    storage_C_par       :: Vector{Matrix{T}}
    storage_gradC_par   :: Vector{Vector{T}}
end

function TDCModel(
    obs                 :: TDCMObs,
    mu_beta_prior       :: Vector{Vector{T}},
    V_beta_prior        :: Vector{Matrix{T}},
    mu_omega_prior      :: Vector{Vector{Vector{Vector{Vector{T}}}}},
    V_omega_prior       :: Vector{Vector{Vector{Vector{Matrix{T}}}}},
    a_tau_prior         :: Vector{Vector{Vector{Vector{T}}}},
    b_tau_prior         :: Vector{Vector{Vector{Vector{T}}}},
    M                   :: Int;
    # This option allocates extra memory based on number of threads availible in the environment
    enable_parallel     :: Bool=false
) where T <: AbstractFloat
    # Number of students, time points, questions, skills, attribute profiles, groups
    N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)
    # Initialize variational distribution parameters
    pi_star = Vector{Vector{Vector{T}}}(undef, N)
    for i in 1:N
        pi_star[i] = Vector{Vector{T}}(undef, O)
        for t in 1:O
            # probability vector of possible single skill mastery over all time points
            pi_star[i][t] = ones(2^K) ./ 2^K
        end
    end
    mu_beta_star = Vector{Vector{T}}(undef, J)
    V_beta_star = Vector{Matrix{T}}(undef, J)
    for j in 1:J
        num_features = size(obs.D[j], 2)
        mu_beta_star[j] = zeros(num_features)
        V_beta_star[j] = Matrix(1.0I, num_features, num_features)
    end
    mu_gamma_star = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, K)
    V_gamma_star = Vector{Vector{Vector{Vector{Matrix{T}}}}}(undef, K)
    for k in 1:K
        mu_gamma_star[k] = Vector{Vector{Vector{Vector{T}}}}(undef, O)
        V_gamma_star[k] = Vector{Vector{Vector{Matrix{T}}}}(undef, O)
        for t in 1:O
            if t == 1
                mu_gamma_star[k][t] = Vector{Vector{Vector{T}}}(undef, 1)
                V_gamma_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 1)

                mu_gamma_star[k][t][1] = Vector{Vector{T}}(undef, S)
                V_gamma_star[k][t][1] = Vector{Matrix{T}}(undef, S)
                for s in 1:S
                    mu_gamma_star[k][t][1][s] = zeros(1)
                    V_gamma_star[k][t][1][s] = ones(1, 1)
                end
            else
                mu_gamma_star[k][t] = Vector{Vector{Vector{T}}}(undef, 2)
                V_gamma_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 2)
                num_features = size(obs.X[k][t], 2)
                for z in 1:2
                    mu_gamma_star[k][t][z] = Vector{Vector{T}}(undef, S)
                    V_gamma_star[k][t][z] = Vector{Matrix{T}}(undef, S)
                    for s in 1:S
                        mu_gamma_star[k][t][z][s] = zeros(num_features)
                        V_gamma_star[k][t][z][s] = Matrix(1.0I, num_features, num_features)
                    end
                end
            end
        end
    end
    mu_omega_star = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, K)
    V_omega_star = Vector{Vector{Vector{Vector{Matrix{T}}}}}(undef, K)
    a_tau_star = Vector{Vector{Vector{Vector{T}}}}(undef, K)
    b_tau_star = Vector{Vector{Vector{Vector{T}}}}(undef, K)
    for k in 1:K
        mu_omega_star[k] = Vector{Vector{Vector{Vector{T}}}}(undef, O)
        V_omega_star[k] = Vector{Vector{Vector{Matrix{T}}}}(undef, O)
        a_tau_star[k] = Vector{Vector{Vector{T}}}(undef, O)
        b_tau_star[k] = Vector{Vector{Vector{T}}}(undef, O)
        for t in 1:O
            num_features_gamma = size(obs.X[k][t], 2)
            num_features_omega = size(obs.U[k][t], 2)
            if t == 1
                mu_omega_star[k][t] = Vector{Vector{Vector{T}}}(undef, 1)
                mu_omega_star[k][t][1] = Vector{Vector{T}}(undef, 1)
                mu_omega_star[k][t][1][1] = zeros(num_features_omega)

                V_omega_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 1)
                V_omega_star[k][t][1] = Vector{Matrix{T}}(undef, 1)
                V_omega_star[k][t][1][1] = Matrix{T}(1.0I, num_features_omega, num_features_omega)

                a_tau_star[k][t] = Vector{Vector{T}}(undef, 1)
                a_tau_star[k][t][1] = ones(1)

                b_tau_star[k][t] = Vector{Vector{T}}(undef, 1)
                b_tau_star[k][t][1] = ones(1)
            else
                mu_omega_star[k][t] = Vector{Vector{Vector{T}}}(undef, 2)
                V_omega_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 2)
                a_tau_star[k][t] = Vector{Vector{T}}(undef, 2)
                a_tau_star[k][t] = Vector{Vector{T}}(undef, 2)
                for z in 1:2
                    mu_omega_star[k][t][z] = Vector{Vector{T}}(undef, num_features_gamma)
                    V_omega_star[k][t][z] = Vector{Matrix{T}}(undef, num_features_gamma)
                    a_tau_star[k][t][z] = ones(num_features_gamma) .* 3
                    a_tau_star[k][t][z] = ones(num_features_gamma) .* 3
                    for m in 1:num_features_gamma
                        mu_omega_star[k][t][z][m] = zeros(num_features_omega)
                        V_omega_star[k][t][z][m] = Matrix(1.0I, num_features_omega, num_features_omega)
                    end
                end
            end
        end
    end
    # Preallocate space for samples from variational distribution
    Z_sample = Vector{Vector{Vector{Vector{Int}}}}(undef, N)
    for i in 1:N
        Z_sample[i] = Vector{Vector{Vector{Int}}}(undef, O)
        for t in 1:O
            Z_sample[i][t] = Vector{Vector{Int}}(undef, M)
            for m in 1:M
                Z_sample[i][t][m] = Vector{Int}(undef, 2^(K))
            end
        end
    end
    beta_sample = Vector{Vector{Vector{T}}}(undef, J)
    for j in 1:J
        beta_sample[j] = Vector{Vector{T}}(undef, M)
        num_features = size(obs.D[j], 2)
        for m in 1:M
            beta_sample[j][m] = Vector{T}(undef, num_features)
        end
    end
    gamma_sample = Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}(undef, K)
    for k in 1:K
        gamma_sample[k] = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, O)
        for t in 1:O
            if t == 1
                gamma_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 1)
                gamma_sample[k][t][1] = Vector{Vector{Vector{T}}}(undef, S)
                for s in 1:S
                    gamma_sample[k][t][1][s] = Vector{Vector{T}}(undef, M)
                    for m in 1:M
                        gamma_sample[k][t][1][s][m] = Vector{T}(undef, 1)
                    end
                end
            else
                num_features = size(obs.X[k][t], 2)
                gamma_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 2)
                for z in 1:2
                    gamma_sample[k][t][z] = Vector{Vector{Vector{T}}}(undef, S)
                    for s in 1:S
                        gamma_sample[k][t][z][s] = Vector{Vector{T}}(undef, M)
                        for m in 1:M
                            gamma_sample[k][t][z][s][m] = Vector{T}(undef, num_features)
                        end
                    end
                end
            end
        end
    end
    omega_sample = Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}(undef, K)
    tau_sample = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, K)
    for k in 1:K
        omega_sample[k] = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, O)
        tau_sample[k] = Vector{Vector{Vector{Vector{T}}}}(undef, O)
        for t in 1:O
            num_features_gamma = size(obs.X[k][t], 2)
            num_features_omega = size(obs.U[k][t], 2)
            if t == 1
                omega_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 1)
                omega_sample[k][t][1] = Vector{Vector{Vector{T}}}(undef, 1)
                omega_sample[k][t][1][1] = Vector{Vector{T}}(undef, M)

                tau_sample[k][t] = Vector{Vector{Vector{T}}}(undef, 1)
                tau_sample[k][t][1] = Vector{Vector{T}}(undef, 1)
                tau_sample[k][t][1][1] = Vector{T}(undef, M)
                for m in 1:M
                    omega_sample[k][t][1][1][m] = Vector{T}(undef, num_features_omega)
                end
            else
                omega_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 2)
                tau_sample[k][t] = Vector{Vector{Vector{T}}}(undef, 2)
                for z in 1:2
                    omega_sample[k][t][z] = Vector{Vector{Vector{T}}}(undef, num_features_gamma)
                    tau_sample[k][t][z] = Vector{Vector{T}}(undef, num_features_gamma)
                    for g in 1:num_features_gamma
                        omega_sample[k][t][z][g] = Vector{Vector{T}}(undef, M)
                        tau_sample[k][t][z][g] = Vector{T}(undef, M)
                        for m in 1:M
                            omega_sample[k][t][z][g][m] = Vector{T}(undef, num_features_omega)
                        end
                    end
                end
            end
        end
    end
    # Preallocate storage for noisy gradient descent calculations
    storage_L = Vector{T}(undef, L)
    storage_L2 = similar(storage_L)
    storage_L3 = similar(storage_L)
    storage_L4 = similar(storage_L)
    storage_LL = Matrix{T}(undef, L, L)
    storage_LL2 = similar(storage_LL)
    storage_LL3 = similar(storage_LL)
    storage_LL4 = similar(storage_LL)
    # Preallocate storage for matrix vectorization operations
    storage_comm = Matrix{T}(undef, L^2, L^2)
    storage_dup = Matrix{T}(undef, L^2, Int(L*(L+1)/2))
    storage_Lsqr = Vector{T}(undef, L^2)
    storage_Lsqr2 = Vector{T}(undef, L^2)
    storage_L2L2 = Matrix{T}(undef, L^2, L^2)
    storage_C = Matrix{T}(undef, L, L)
    storage_gradC = Vector{T}(undef, Int(L*(L+1)/2))
    # Preallocate Identity matrix
    I_LL = Matrix{T}(I, L, L)
    # Allocate optional space for parallel computing
    nthreads = Threads.nthreads()
    storage_L_par = Vector{Vector{T}}(undef, nthreads)
    storage_L2_par = similar(storage_L_par)
    storage_L3_par = similar(storage_L_par)
    storage_L4_par = similar(storage_L_par)
    storage_LL_par = Vector{Matrix{T}}(undef, nthreads)
    storage_LL2_par = similar(storage_LL_par)
    storage_LL3_par = similar(storage_LL_par)
    storage_LL4_par = similar(storage_LL_par)
    storage_comm_par = Vector{Matrix{T}}(undef, nthreads)
    storage_dup_par = Vector{Matrix{T}}(undef, nthreads)
    storage_Lsqr_par = Vector{Vector{T}}(undef, nthreads)
    storage_Lsqr2_par = Vector{Vector{T}}(undef, nthreads)
    storage_L2L2_par = Vector{Matrix{T}}(undef, nthreads)
    storage_C_par = Vector{Matrix{T}}(undef, nthreads)
    storage_gradC_par = Vector{Vector{T}}(undef, nthreads)
    if enable_parallel
        storage_L_par[1] = storage_L
        storage_L2_par[1] = storage_L2
        storage_L3_par[1] = storage_L3
        storage_L4_par[1] = storage_L4
        storage_LL_par[1] = storage_LL
        storage_LL2_par[1] = storage_LL2
        storage_LL3_par[1] = storage_LL3
        storage_LL4_par[1] = storage_LL4
        storage_comm_par[1] = storage_comm
        storage_dup_par[1] = storage_dup
        storage_Lsqr_par[1] = storage_Lsqr
        storage_Lsqr2_par[1] = storage_Lsqr2
        storage_L2L2_par[1] = storage_L2L2
        storage_C_par[1] = storage_C
        storage_gradC_par[1] = storage_gradC_par
        for thread in 2:nthreads
            storage_L_par[thread] = Vector{T}(undef, L)
            storage_L2_par[thread] = similar(storage_L)
            storage_L3_par[thread] = similar(storage_L)
            storage_L4_par[thread] = similar(storage_L)
            storage_LL_par[thread] = Matrix{T}(undef, L, L)
            storage_LL2_par[thread] = similar(storage_LL)
            storage_LL3_par[thread] = similar(storage_LL)
            storage_LL4_par[thread] = similar(storage_LL)
            storage_comm_par[thread] = Matrix{T}(undef, L^2, L^2)
            storage_dup_par[thread] = Matrix{T}(undef, L^2, Int(L*(L+1)/2))
            storage_Lsqr_par[thread] = Vector{T}(undef, L^2)
            storage_Lsqr2_par[thread] = Vector{T}(undef, L^2)
            storage_L2L2_par[thread] = Matrix{T}(undef, L^2, L^2)
            storage_C_par[thread] = Matrix{T}(undef, L, L)
            storage_gradC_par[thread] = Vector{T}(undef, Int(L*(L+1)/2))
        end
        println("TDCModel constructed for computation on $nthreads threads")
    end
    # Initialize DCModel object
    TDCModel(obs, mu_beta_prior, V_beta_prior, mu_omega_prior, V_omega_prior, a_tau_prior, b_tau_prior, enable_parallel,
    pi_star, mu_beta_star, V_beta_star, mu_gamma_star, V_gamma_star, mu_omega_star, V_omega_star, a_tau_star, b_tau_star, M,
    Z_sample, beta_sample, gamma_sample, omega_sample, tau_sample,
    storage_L, storage_L2, storage_L3, storage_L4, storage_LL, storage_LL2, storage_LL3, storage_LL4,
    storage_comm, storage_dup, storage_Lsqr, storage_Lsqr2, storage_L2L2, storage_C, storage_gradC,
    I_LL, storage_L_par, storage_L2_par, storage_L3_par, storage_L4_par, storage_LL_par, storage_LL2_par, storage_LL3_par, storage_LL4_par,
    storage_comm_par, storage_dup_par, storage_Lsqr_par, storage_Lsqr2_par, storage_L2L2_par, storage_C_par, storage_gradC_par)
end

TDCModel

In [22]:
function sample_Z(
    model           :: TDCModel,
    idx_student     :: Int,
    idx_time        :: Int
)
    obs = model.obs
    # Create variational distribution from model parameters for specific Z
    Z_it_variational_distribution = Multinomial(1, model.pi_star[idx_student][idx_time])
    # Populate preallocated arrays with samples from variational distribution
    rand!(Z_it_variational_distribution, model.Z_sample[idx_student][idx_time])
end

function sample_β(
    model           :: TDCModel;
    idx_question    :: Int = -1
)
    obs = model.obs
    J = size(obs.Y, 3)
    if idx_question == -1
        for j in 1:J
            # Create variational distribution from model parameters for each β_j
            beta_j_variational_distribution = MvNormal(model.mu_beta_star[j], model.V_beta_star[j])
            # Populate preallocated arrays with samples from variational distribution
            rand!(beta_j_variational_distribution, model.beta_sample[j])
        end
    else
        # Create variational distribution from model parameters for specific β_j
        beta_j_variational_distribution = MvNormal(model.mu_beta_star[idx_question], model.V_beta_star[idx_question])
        # Populate preallocated arrays with samples from variational distribution
        rand!(beta_j_variational_distribution, model.beta_sample[idx_question])
    end
end

function sample_γ(
    model           :: TDCModel,
    idx_group       :: Int,
    idx_time        :: Int,
    idx_skill       :: Int,
    indicator_skill :: Int
)
    # Create variational distribution from model parameters for γ
    gamma_stkz_variational_distribution = MvNormal(model.mu_gamma_star[idx_skill][idx_time][indicator_skill + 1][idx_group],
                                            model.V_gamma_star[idx_skill][idx_time][indicator_skill + 1][idx_group])
    # Populate preallocated arrays with samples from variational distribution
    rand!(gamma_stkz_variational_distribution, model.gamma_sample[idx_skill][idx_time][indicator_skill + 1][idx_group])
end
;

In [26]:
function update_categorical_variational_distribution(
    model               :: TDCModel;
    step                :: T = 1e-2,
    tol                 :: T = 1e-6,
    maxiter             :: Int = 100000,
    verbose             :: Bool = true
) where T <: AbstractFloat
    obs = model.obs
    Y, D, X = obs.Y, obs.D, obs.X
    Z_sample, beta_sample, gamma_sample = model.Z_sample, model.beta_sample, model.gamma_sample
    pi_star_old = model.pi_star
    # Number of students, time points, questions, skills, attribute profiles, groups
    N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)
    M = model.M
    # Fully update parameters of each Z_i using noisy gradients before moving to update parameters of next Z_i
    if !model.enable_parallel
        @inbounds for i in 1:N
            # Storage for gradient terms
            grad_log_q = model.storage_L2
            grad_L = model.storage_L3
            # Storage for intermediate term in gradient calculations
            D_beta = model.storage_L
            rho_star_old_i = view(model.storage_LL3, 1:L)
            # Get parameters for variational distribution of skill of i-th student
            pi_star_old_i = pi_star_old[i][1]
            # Get group number of student i
            group_i = obs.group[i]
            # Perform gradient descent update of i-th π*    
            @inbounds for iter in 1:maxiter
                # Rho is unique up to a constant addative term
                rho_star_old_i = log.(pi_star_old_i)
                # Sample Z with updated π*
                sample_Z(model, i, 1)
                # Set gradient of ELBO to 0
                fill!(grad_L, 0)
                # Rao Blackwellized ELBO
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                @inbounds for m in 1:M
                    z_im = Z_sample[i][1][m]
                    # Calculate gradient of log(q_1i(Z_i)) w.r.t. π*_i
                    grad_log_q .= z_im .- pi_star_old_i
                    # Calculate log(p(Y, Z_(i)))
                    log_prob_YZ = 0
                    for j in 1:J
                        mul!(D_beta, D[j], beta_sample[j][m])
                        log_prob_YZ += dot(z_im, log.(sigmoid.((2*Y[i,j] - 1) .* D_beta)))
                    end
                    skill_profile = obs.skill_dict[argmax(z_im)]
                    for k in 1:K
                        log_prob_YZ += log(sigmoid((2*skill_profile[k] - 1) * dot(gamma_sample[k][1][1][group_i][m], obs.X[k][1][i])))
                    end
                    # Calculate log(q_1i(Z_i))
                    log_q = dot(z_im, log.(pi_star_old_i))
                    # Update average gradient
                    grad_L .= (m - 1)/m .* grad_L + 1/m .* grad_log_q .* (log_prob_YZ - log_q)
                    # Update ELBO estimator
                    ELBO = (m-1)/m * ELBO + 1/m * (log_prob_YZ - log_q)
                end
                # Print ELBO, parameter and gradient if verbose
                if verbose
                    println("ELBO: $ELBO")
                    println("π*_$i: $pi_star_old_i")
                    println("gradient: $grad_L")
                end
                # Update with one step
                rho_star_old_i .+= step * grad_L
                # Convert logits into probabilities
                pi_star_old_i .= exp.(rho_star_old_i) ./ sum(exp.(rho_star_old_i))
                # Stop condition
                if abs2(norm(grad_L)) <= tol
                    break
                end
            end
        end
    else
        Threads.@threads for i in 1:N
            # Get Thread Id
            tid = Threads.threadid()
            # Storage for gradient terms
            grad_log_q = model.storage_L2_par[tid]
            grad_L = model.storage_L3_par[tid]
            # Storage for intermediate term in gradient calculations
            D_beta = model.storage_L_par[tid]
            rho_star_old_i = view(model.storage_LL3_par[tid], 1:L)
            # Get parameters for variational distribution of skill of i-th student
            pi_star_old_i = pi_star_old[i]
            # Perform gradient descent update of i-th π*    
            @inbounds for iter in 1:maxiter
                # Rho is unique up to a constant addative term
                rho_star_old_i = log.(pi_star_old_i)
                # Sample Z with updated π*
                sample_variational_distribution(model, sample_Z = true, idx_Z = i)
                # Set gradient of ELBO to 0
                fill!(grad_L, 0)
                # Rao Blackwellized ELBO
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                @inbounds for m in 1:M
                    z_im = Z_sample[i][m]
                    # Calculate gradient of log(q_1i(Z_i)) w.r.t. π*_i
                    grad_log_q .= z_im .- pi_star_old_i
                    # Calculate log(p(Y, Z_(i)))
                    log_prob_YZ = 0
                    for j in 1:J
                        mul!(D_beta, D[j], beta_sample[j][m])
                        log_prob_YZ += dot(z_im, log.(sigmoid.((2*Y[i,j] - 1) .* D_beta)))
                    end
                    log_prob_YZ += dot(z_im, log.(pi_sample[m]))
                    # Calculate log(q_1i(Z_i))
                    log_q = dot(z_im, log.(pi_star_old_i))
                    # Update average gradient
                    grad_L .= (m - 1)/m .* grad_L + 1/m .* grad_log_q .* (log_prob_YZ - log_q)
                    # Update ELBO estimator
                    ELBO = (m-1)/m * ELBO + 1/m * (log_prob_YZ - log_q)
                end
                # Print ELBO, parameter and gradient if verbose
                if verbose
                    println("ELBO: $ELBO")
                    println("π*_$i: $pi_star_old_i")
                    println("gradient: $grad_L")
                end
                # Update with one step
                rho_star_old_i .+= step * grad_L
                # Convert logits into probabilities
                pi_star_old_i .= exp.(rho_star_old_i) ./ sum(exp.(rho_star_old_i))
                # Stop condition
                if abs2(norm(grad_L)) <= tol
                    break
                end
            end
        end
    end
end

update_categorical_variational_distribution (generic function with 1 method)

In [3]:
using RCall

R"""
load("TDCM_multilevel_data.RData")
"""
TDCM_data = @rget data
Y = Array{Int, 3}(TDCM_data[:Y])
Q = convert(Matrix{Int64}, TDCM_data[:Q_matrix])
U = Vector{Vector{Matrix{Float64}}}(TDCM_data[:X_group])
for skill in TDCM_data[:X_ind]
    for time in 1:length(skill)
        if skill[time] isa Vector{<: Number}
            skill[time] = reshape(skill[time], :, 1)
        end
    end
end
X = Vector{Vector{Matrix{Float64}}}(TDCM_data[:X_ind])
group = Vector{Int64}(TDCM_data[:group])
obs = TDCMObs(Y, Q, U, X, group)
;

In [53]:
N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)

mu_beta_prior = Vector{Vector{Float64}}(undef, J)
V_beta_prior = Vector{Matrix{Float64}}(undef, J)
for j in 1:J
    num_features = size(obs.D[j], 2)
    mu_beta_prior[j] = zeros(num_features)
    V_beta_prior[j] = Matrix(1.0I, num_features, num_features)
end

mu_omega_prior = Vector{Vector{Vector{Vector{Vector{Float64}}}}}(undef, K)
V_omega_prior = Vector{Vector{Vector{Vector{Matrix{Float64}}}}}(undef, K)
a_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)
b_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)

for k in 1:K
    mu_omega_prior[k] = Vector{Vector{Vector{Vector{Float64}}}}(undef, O)
    V_omega_prior[k] = Vector{Vector{Vector{Matrix{Float64}}}}(undef, O)
    a_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    b_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    for t in 1:O
        num_features_gamma = size(obs.X[k][t], 2)
        num_features_omega = size(obs.U[k][t], 2)
        if t == 1
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 1)
            mu_omega_prior[k][t][1] = Vector{Vector{Float64}}(undef, 1)
            mu_omega_prior[k][t][1][1] = zeros(num_features_omega)

            V_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 1)
            V_omega_prior[k][t][1] = Vector{Matrix{Float64}}(undef, 1)
            V_omega_prior[k][t][1][1] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)

            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            a_tau_prior[k][t][1] = ones(1)

            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            b_tau_prior[k][t][1] = ones(1)
        else
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 2)
            V_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 2)
            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            for z in 1:2
                mu_omega_prior[k][t][z] = Vector{Vector{Float64}}(undef, num_features_gamma)
                V_omega_prior[k][t][z] = Vector{Matrix{Float64}}(undef, num_features_gamma)
                a_tau_prior[k][t][z] = ones(num_features_gamma) .* 3
                b_tau_prior[k][t][z] = ones(num_features_gamma) .* 3
                for m in 1:num_features_gamma
                    mu_omega_prior[k][t][z][m] = zeros(num_features_omega)
                    V_omega_prior[k][t][z][m] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)
                end
            end
        end
    end
end

M = 1000
model = TDCModel(obs, mu_beta_prior, V_beta_prior, mu_omega_prior, V_omega_prior, a_tau_prior, b_tau_prior, M);

**Estimating attribute profiles and betas**

In [54]:
# Fix true values of gamma and betas
for k in 1:K
    for s in S
        model.mu_gamma_star[k][1][1][s] .= data[:gamma][k][1][s]
        model.V_gamma_star[k][1][1][s] = model.V_gamma_star[k][1][1][s] ./ 100
    end
end
for j in 1:J
    model.mu_beta_star[j] = data[:beta][j]
    model.V_beta_star[j] = model.V_beta_star[j] ./ 100
end

# Sample beta and gammas
sample_β(model)
for k in 1:K
    for t in 1:O
        if t == 1
            for s in 1:S
                sample_γ(model, s, t, k, 0)
            end
        else
            for z in 0:1
                for s in 1:S
                    sample_γ(model, s, t, k, z)
                end
            end
        end
    end
end

In [55]:
update_categorical_variational_distribution(model, maxiter=200, verbose=false)

In [70]:
skill_profiles = Dict(1=>[0, 0], 
                 2=>[0, 1],
                 3=>[1, 0],
                 4=>[1, 1])

pred = []
accuracy = 0
for i in 1:5000
    push!(pred, skill_profiles[argmax(model.pi_star[i][1])])
    correct = all(skill_profiles[argmax(model.pi_star[i][1])] .== Vector{Int}(data[:profiles][i, 1, :]))
    accuracy += correct
    if !correct
        print(Vector{Int}(data[:profiles][i, 1, :]))
        println(skill_profiles[argmax(model.pi_star[i][1])])
    end
end

accuracy/5000

[0, 1][0, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[0, 1][1, 0]
[0, 0][0, 1]
[1, 0][0, 0]
[0, 0][1, 0]
[1, 1][0, 1]
[0, 1][1, 1]
[0, 0][1, 0]
[0, 0][1, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[0, 0][1, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[1, 0][0, 1]
[0, 1][0, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][1, 0]
[1, 0][0, 1]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 0][1, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][1, 1]
[1, 1][1, 0]
[0, 1][1, 0]
[0, 1][0, 0]
[1, 1][0, 1]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[0, 0][1, 0]
[1, 1][0, 1]
[0, 0][1, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[0, 1][1, 1]
[0, 1][1, 1]
[0, 0][1, 0]
[0, 0][1, 0]
[0, 1][0, 0]
[0, 1][1, 0]
[0, 1][0, 0]
[0, 0][1, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[0, 1][0, 0]
[0, 1][1, 1]
[1, 0][0, 1]
[1, 0][0, 0]
[1, 0][0, 0]
[1, 0][0, 0]
[0, 1][0, 0]
[1, 0][0, 0]
[1, 1][0, 1]
[1, 0][0, 0]
[0, 0][1, 0]
[1, 0][0, 0]
[1, 1][0, 1]
[1, 1][0, 1]

0.7298

In [64]:
model.pi_star

5000-element Vector{Vector{Vector{Float64}}}:
 [[0.15421050185690477, 0.4460410059727272, 0.3603976897568462, 0.039350802413521783], [0.25, 0.25, 0.25, 0.25]]
 [[0.010919896548400749, 0.039771360759159095, 0.03549297843222924, 0.9138157642602108], [0.25, 0.25, 0.25, 0.25]]
 [[0.9517948881998806, 0.01704634371815943, 0.025154787624442534, 0.006003980457517625], [0.25, 0.25, 0.25, 0.25]]
 [[0.9693058033965382, 0.015536281878567468, 0.01114661648879353, 0.004011298236100831], [0.25, 0.25, 0.25, 0.25]]
 [[0.9749739337854569, 0.00950087781009139, 0.012001991447073264, 0.003523196957378355], [0.25, 0.25, 0.25, 0.25]]
 [[0.041067770444428046, 0.8406002767455435, 0.07192697477934556, 0.04640497803068302], [0.25, 0.25, 0.25, 0.25]]
 [[0.9298672138807119, 0.02324799215328718, 0.03956347878298722, 0.007321315183013599], [0.25, 0.25, 0.25, 0.25]]
 [[0.5877778627449293, 0.35859509230938513, 0.03742024645720164, 0.01620679848848404], [0.25, 0.25, 0.25, 0.25]]
 [[0.21980319569261306, 0.02484569241586

In [68]:
model.mu_beta_star

25-element Vector{Vector{Float64}}:
 [-2.5, 4.5]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 4.5]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 4.5]
 [-2.5, 4.5]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 4.5]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 4.5]
 [-2.5, 4.5]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 4.5]
 [-2.5, 4.5]
 [-2.5, 4.5]
 [-2.5, 4.5]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 4.5]
 [-2.5, 4.5]
 [-2.5, 4.5]
 [-2.5, 2.0, 2.0, 1.0]
 [-2.5, 4.5]
 [-2.5, 4.5]

In [69]:
model.beta_sample

25-element Vector{Vector{Vector{Float64}}}:
 [[-2.5737656606768717, 4.520929120344847], [-2.6646604024399463, 4.439556582440321], [-2.604852149763464, 4.595093512223326], [-2.557808827917075, 4.6228719523666655], [-2.4699363966174177, 4.5512716881233395], [-2.5399061638405116, 4.4531040799732295], [-2.4588459704212524, 4.455569470125392], [-2.5824008831505436, 4.475664075193032], [-2.5200972780983246, 4.462607413582707], [-2.4157706178345038, 4.428087782468305]  …  [-2.6642702146516104, 4.4353607075316015], [-2.442237549828469, 4.558730191692875], [-2.5405804839317554, 4.53823989942094], [-2.4831849694081902, 4.565000153788496], [-2.5508556498963695, 4.277971965609964], [-2.455768020998327, 4.392697423131329], [-2.4725145064315366, 4.477438396192363], [-2.3015084244874076, 4.59499496518943], [-2.3858778330596877, 4.578334470170068], [-2.626174108249534, 4.340295628244254]]
 [[-2.526279214964151, 1.943717630562195, 2.076617009228228, 0.9579664180386562], [-2.6544565411520233, 1.86214216